# Postdam PM2.5 Linear Regression Forcasting 

* With Ridge And Lasso Regression Models
* Between 2013 and 2023, data collected by DEBB021 was used.
* To increase the accuracy of PM2.5 data estimation, NO2, O3, SO2, PM10 pollutant gas data accepted by the EEA was added.


In [2]:
# imports
import os
os.chdir('..')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np, pandas as pd
import src.model_base as mb

## Data Exploration

* Load Data


In [3]:
df = pd.read_csv('data/CLEAN_MERGED_DE_DEBB021.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/CLEAN_MERGED_DE_DEBB021.csv'

In [ ]:
df.describe().transpose()

In [ ]:
df.set_index('Start_Timestamp', inplace=True)
df.head()

# Linear Regression 

* Defining Target and feature variables
* Scale Features
* Apply PCA
* Split Data into Train, Validation, Test data
* Train Linear Reg Model
* Predict
* Evaluate Validation and Test data
* Hyperparameter Tuning ??


In [ ]:
# Defining Target and feature variables
features = ['NO2-Value', 'O3-Value', 'SO2-Value', 'PM10-Value']
target = 'PM2.5-Value'

# Separate the features and target
X = df[features]
y = df[target]

## Splitting Data 

Train, Validation and Test data

In [ ]:
train_ratio = 0.6
validation_ratio = 0.2
test_ratio = 0.2

# Calculate the indices for the splits
train_end = int(len(df) * train_ratio)
validation_end = train_end + int(len(df) * validation_ratio)

# Split the dataset
train_data = df[:train_end]
validation_data = df[train_end:validation_end]
test_data = df[validation_end:]


print(f"Training set size: {train_data.shape[0]}")
print(f"Validation set size: {validation_data.shape[0]}")
print(f"Test set size: {test_data.shape[0]}")


In [ ]:
# Scale the features
scaler = StandardScaler()
scaler.fit(train_data[features])  # Fit only on training data

# Scale the datasets
X_train_scaled = scaler.transform(train_data[features])
X_val_scaled = scaler.transform(validation_data[features])
X_test_scaled = scaler.transform(test_data[features])

## Principle Component Analysis (PCA)
Principal Component Analysis (PCA) is a statistical procedure that uses an orthogonal transformation to convert a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables called principal components.

In [ ]:
# Apply PCA on the scaled data
pca = PCA(n_components=0.95)  # Adjust based on the explained variance
pca.fit(X_train_scaled)  # Fit only on training data

In [ ]:
# Transform the datasets using the fitted PCA
X_train_pca = pca.transform(X_train_scaled)
X_val_pca = pca.transform(X_val_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [ ]:
# Extract the target variable
y_train = train_data[target]
y_val = validation_data[target]
y_test = test_data[target]

## Model Creation
* Initialize Linear Regression Model
* Train model

In [ ]:
# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train_pca, y_train)

# Evaluation 

## With Validation Data

Error metrics MAE, MSE, RMSE, MASE, MAPE

* Regarding the MASE metric, calculating it requires a baseline prediction model for the time series, which is typically done by using the last observed value to predict the next (in the simplest case) or using more complex methods like ARIMA for one-step ahead forecasting. This is not included in the above script as it would require additional steps to implement the naive forecasting method for a time series.

### Predict Validation


In [ ]:
# Make predictions on the validation set
y_val_pred = model.predict(X_val_pca)

print(y_val_pred)


In [ ]:
# Error Metric
mae_val = mean_absolute_error(y_val, y_val_pred)
mse_val = mean_squared_error(y_val, y_val_pred)
rmse_val = mse_val ** 0.5
mape_val = mean_absolute_percentage_error(y_val, y_val_pred)

print(f"Validation MAE: {mae_val}")
print(f"Validation MSE: {mse_val}")
print(f"Validation RMSE: {rmse_val}")
print(f"Validation MAPE: {mape_val}")

In [ ]:
# MASE Calculation

# Create naive forecasts for validation and test sets
y_val_naive = np.roll(y_val, 1)

# Calculate MAE for the naive forecasts
mae_val_naive = mean_absolute_error(y_val[1:], y_val_naive[1:])

# Calculate MASE for validation and test sets
mase_val = mae_val / mae_val_naive

print(f"Validation MASE: {mase_val}")

## With Test Data


In [ ]:
# Predict on the test set
y_test_pred = model.predict(X_test_pca)

# Calculate and print the test metrics
mae_test = mean_absolute_error(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = mse_test ** 0.5
mape_test = mean_absolute_percentage_error(y_test, y_test_pred)

print(f"\nTest MAE: {mae_test}")
print(f"Test MSE: {mse_test}")
print(f"Test RMSE: {rmse_test}")
print(f"Test MAPE: {mape_test}")

In [ ]:
# MASE Calculation

# Create naive forecasts for validation and test sets
y_test_naive = np.roll(y_test, 1)

# Calculate MAE for the naive forecasts
mae_test_naive = mean_absolute_error(y_test[1:], y_test_naive[1:])

# Calculate MASE for validation and test sets
mase_test = mae_test / mae_test_naive

print(f"Test MASE: {mase_test}")

## Plot Table 


In [ ]:

# Let's assume y_val is your validation set actual values and y_val_pred are the predictions
# Similarly, y_test and y_test_pred for the test set

# Plotting the actual vs predicted values for validation set
plt.figure(figsize=(15, 5))

# Actual values - using blue color with a line marker
plt.plot(validation_data.index, y_val, color='blue', marker='o', label='Actual', linestyle='-', linewidth=1)

# Predicted values - using red color with a cross marker
plt.plot(validation_data.index, y_val_pred, color='red', marker='x', label='Predicted', linestyle='None')

plt.title('Validation Set - Actual vs Predicted PM2.5')
plt.xlabel('Date')
plt.ylabel('PM2.5')
plt.legend()
plt.show()

# For the test set, do similarly
plt.figure(figsize=(15, 5))

# Actual values - using blue color with a line marker
plt.plot(test_data.index, y_test, color='blue', marker='o', label='Actual', linestyle='-', linewidth=1)

# Predicted values - using red color with a cross marker
plt.plot(test_data.index, y_test_pred, color='red', marker='x', label='Predicted', linestyle='None')

plt.title('Test Set - Actual vs Predicted PM2.5')
plt.xlabel('Date')
plt.ylabel('PM2.5')
plt.legend()
plt.show()

# HyperPramater Tuning

Linear Regression typically has fewer hyperparameters than other models like neural networks or ensemble models. However, there are still some aspects of the model that you can adjust. For instance, you can apply regularization, which can be considered a form of hyperparameter tuning. The most common types of regularized linear regression are Ridge Regression (L2 regularization) and Lasso Regression (L1 regularization).

## Ridge Regression (L2)
Ridge regression, also known as Tikhonov regularization, is a method of estimating the coefficients of multiple-regression models in scenarios where linearly independent variables are highly correlated. In multicollinear data, least-squares estimates are unbiased, but their variances are large, so they may be far from the true value. Ridge regression adds a degree of bias to the regression estimates, which often results in a decrease in the standard errors.

* RandomizedSearchCV is a useful alternative to GridSearchCV when the parameter space is large. It samples a fixed number of parameter combinations from the specified distributions, which can be much more efficient, especially when some hyperparameters do not influence the performance of the model significantly.


### Model Creation

In [ ]:
# Define the model
model = Ridge()


In [ ]:
# Define the distribution of hyperparameters to sample from
# Here, we define a uniform distribution over a log scale for the alpha parameter
param_distributions = {'alpha': uniform(1e-4, 1e4)}

# Set up the random search with cross-validation
# n_iter sets the number of different combinations to try
# cv is the number of folds for cross-validation
# scoring determines the metric used to evaluate the models
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, n_iter=100, cv=5, scoring='neg_mean_absolute_error', verbose=1, random_state=42, n_jobs=-1)

### Model Train

In [ ]:
# Fit the random search to the data (use the PCA-transformed data)
random_search.fit(X_train_pca, y_train)

# Print the best parameters
print('Best parameters found: ', random_search.best_params_)

In [ ]:
# Use the best estimator to make predictions
y_val_pred = random_search.best_estimator_.predict(X_val_pca)
y_test_pred = random_search.best_estimator_.predict(X_test_pca)

In [ ]:
# Validation Set

# Error Metric
mae_val = mean_absolute_error(y_val, y_val_pred)
mse_val = mean_squared_error(y_val, y_val_pred)
rmse_val = mse_val ** 0.5
mape_val = mean_absolute_percentage_error(y_val, y_val_pred)

print(f"Validation MAE: {mae_val}")
print(f"Validation MSE: {mse_val}")
print(f"Validation RMSE: {rmse_val}")
print(f"Validation MAPE: {mape_val}")

# MASE Calculation

# Create naive forecasts for validation and test sets
y_val_naive = np.roll(y_val, 1)

# Calculate MAE for the naive forecasts
mae_val_naive = mean_absolute_error(y_val[1:], y_val_naive[1:])

# Calculate MASE for validation and test sets
mase_val = mae_val / mae_val_naive

print(f"Validation MASE: {mase_val}")


In [ ]:
# Test Set
# Calculate and print the test metrics
mae_test = mean_absolute_error(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = mse_test ** 0.5
mape_test = mean_absolute_percentage_error(y_test, y_test_pred)

print(f"\nTest MAE: {mae_test}")
print(f"Test MSE: {mse_test}")
print(f"Test RMSE: {rmse_test}")
print(f"Test MAPE: {mape_test}")

# MASE Calculation

# Create naive forecasts for validation and test sets
y_test_naive = np.roll(y_test, 1)

# Calculate MAE for the naive forecasts
mae_test_naive = mean_absolute_error(y_test[1:], y_test_naive[1:])

# Calculate MASE for validation and test sets
mase_test = mae_test / mae_test_naive

print(f"Test MASE: {mase_test}")


In [ ]:

# Let's assume y_val is your validation set actual values and y_val_pred are the predictions
# Similarly, y_test and y_test_pred for the test set

# Plotting the actual vs predicted values for validation set
plt.figure(figsize=(15, 5))

# Actual values - using blue color with a line marker
plt.plot(validation_data.index, y_val, color='blue', marker='o', label='Actual', linestyle='-', linewidth=1)

# Predicted values - using red color with a cross marker
plt.plot(validation_data.index, y_val_pred, color='red', marker='x', label='Predicted', linestyle='None')

plt.title('Validation Set - Actual vs Predicted PM2.5')
plt.xlabel('Date')
plt.ylabel('PM2.5')
plt.legend()
plt.show()

# For the test set, do similarly
plt.figure(figsize=(15, 5))

# Actual values - using blue color with a line marker
plt.plot(test_data.index, y_test, color='blue', marker='o', label='Actual', linestyle='-', linewidth=1)

# Predicted values - using red color with a cross marker
plt.plot(test_data.index, y_test_pred, color='red', marker='x', label='Predicted', linestyle='None')

plt.title('Test Set - Actual vs Predicted PM2.5')
plt.xlabel('Date')
plt.ylabel('PM2.5')
plt.legend()
plt.show()

## Lasso Regression (L1)

Lasso regression, which stands for Least Absolute Shrinkage and Selection Operator, is a type of linear regression that uses shrinkage. Shrinkage is where data values are shrunk towards a central point, like the mean. The lasso procedure encourages simple, sparse models (i.e., models with fewer parameters). This is particularly useful when you have a large number of predictors because it automatically performs variable selection.

The key difference between ridge regression and lasso regression lies in the penalty term added to the cost function. While ridge regression adds a squared magnitude of the coefficient as a penalty term to the cost function, lasso regression adds the absolute value of the magnitude of the coefficient as the penalty term.

The advantages of lasso regression include:

* It can produce simpler and more interpretable models due to feature selection.
* It is less prone to overfitting as it penalizes the absolute size of the coefficients.
* It can handle complex scenarios with many variables or features in the dataset.

However, lasso regression can also have disadvantages, such as biasing estimates towards zero and sometimes underperforming when all variables are included in the true model. It can also struggle with grouped variables, where it tends to select only one variable from a group and ignore the others.

### Model Creation


In [ ]:
# Define the model
model = Lasso(max_iter=10000)  # Increased max_iter for convergence with high-dimensional data

# Define the distribution of hyperparameters to sample from
# loguniform is useful for alpha because it spans several orders of magnitude
param_distributions = {'alpha': loguniform(1e-4, 1e0)}

# Set up the random search with cross-validation
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=100,
    cv=5,
    scoring='neg_mean_absolute_error',
    verbose=1,
    random_state=42,
    n_jobs=-1
)

### Model Training

In [ ]:
# Fit the random search to the data (use the PCA-transformed data)
random_search.fit(X_train_pca, y_train)

# Print the best parameters
print('Best parameters found: ', random_search.best_params_)

# Use the best estimator to make predictions
y_val_pred = random_search.best_estimator_.predict(X_val_pca)
y_test_pred = random_search.best_estimator_.predict(X_test_pca)


In [ ]:
# Validation Set

# Error Metric
mae_val = mean_absolute_error(y_val, y_val_pred)
mse_val = mean_squared_error(y_val, y_val_pred)
rmse_val = mse_val ** 0.5
mape_val = mean_absolute_percentage_error(y_val, y_val_pred)

print(f"Validation MAE: {mae_val}")
print(f"Validation MSE: {mse_val}")
print(f"Validation RMSE: {rmse_val}")
print(f"Validation MAPE: {mape_val}")

# MASE Calculation

# Create naive forecasts for validation and test sets
y_val_naive = np.roll(y_val, 1)

# Calculate MAE for the naive forecasts
mae_val_naive = mean_absolute_error(y_val[1:], y_val_naive[1:])

# Calculate MASE for validation and test sets
mase_val = mae_val / mae_val_naive

print(f"Validation MASE: {mase_val}")

In [ ]:
# Test Set
# Calculate and print the test metrics
mae_test = mean_absolute_error(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = mse_test ** 0.5
mape_test = mean_absolute_percentage_error(y_test, y_test_pred)

print(f"\nTest MAE: {mae_test}")
print(f"Test MSE: {mse_test}")
print(f"Test RMSE: {rmse_test}")
print(f"Test MAPE: {mape_test}")

# MASE Calculation

# Create naive forecasts for validation and test sets
y_test_naive = np.roll(y_test, 1)

# Calculate MAE for the naive forecasts
mae_test_naive = mean_absolute_error(y_test[1:], y_test_naive[1:])

# Calculate MASE for validation and test sets
mase_test = mae_test / mae_test_naive

print(f"Test MASE: {mase_test}")


In [ ]:
# Let's assume y_val is your validation set actual values and y_val_pred are the predictions
# Similarly, y_test and y_test_pred for the test set

# Plotting the actual vs predicted values for validation set
plt.figure(figsize=(15, 5))

# Actual values - using blue color with a line marker
plt.plot(validation_data.index, y_val, color='blue', marker='o', label='Actual', linestyle='-', linewidth=1)

# Predicted values - using red color with a cross marker
plt.plot(validation_data.index, y_val_pred, color='red', marker='x', label='Predicted', linestyle='None')

plt.title('Validation Set - Actual vs Predicted PM2.5')
plt.xlabel('Date')
plt.ylabel('PM2.5')
plt.legend()
plt.show()

# For the test set, do similarly
plt.figure(figsize=(15, 5))

# Actual values - using blue color with a line marker
plt.plot(test_data.index, y_test, color='blue', marker='o', label='Actual', linestyle='-', linewidth=1)

# Predicted values - using red color with a cross marker
plt.plot(test_data.index, y_test_pred, color='red', marker='x', label='Predicted', linestyle='None')

plt.title('Test Set - Actual vs Predicted PM2.5')
plt.xlabel('Date')
plt.ylabel('PM2.5')
plt.legend()
plt.show()